In [0]:
#!pip install eli5
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import cross_val_score

In [0]:
from sklearn.ensemble import RandomForestRegressor
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/dw_Matrix'

/content/drive/My Drive/Colab Notebooks/dw_Matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)


In [0]:

def run_model(feats, model=DecisionTreeRegressor(max_depth=5)): 
  X = df[ feats ].values
  Y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [18]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [19]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [21]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

    #wyrzucanie brudnych danych 
  features = literal_eval(x.replace('\\"', '"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [31]:
#sprawdzamy, ile unikalnych kluczy w zestawie danych
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [55]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
    df [ get_name_feat(key)] = df.features_parsed.map(lambda feats:feats[key] if key in feats else np.nan)

In [56]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_sku', 'feat_clothing size type', 'feat_watch case shape',
       'feat_date first available', 'feat_country of manufacture',
       'feat_labor', 'feat_material', 'feat_spikes type', 'feat_leg opening:',
       'feat_����������������'],
      dtype='object', length=526)

In [60]:
df [ False == df['feat_athlete'].isnull()].shape[0]

8

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] *100

In [71]:
#bierzemy klucze, które wystepuja w przynajmniej 30% przypadków
{k:v for k, v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[ get_name_feat(key) + '_cat'] = df [ get_name_feat(key)].factorize()[0]

In [74]:
#wszystkie rekordy rzutujemy na mala litere i sprawdzamy, ile z brand i feat_brand jest takich samych
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df [df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
feats = ['']

In [76]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.35066547425954, 4.228477075713733)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [102]:
feats=['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model( feats, model)

(-57.06203669054236, 4.296030183872546)

In [93]:
# sprawdzamy sile cech (features) wplywajacych na prognozowana cene
X = df[ feats].values
Y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,Y)

perm = PermutationImportance(m, random_state=1).fit(X,Y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2605 ± 0.0099,brand_cat
0.1070 ± 0.0093,feat_material_cat
0.0452 ± 0.0026,feat_gender_cat
0.0207 ± 0.0011,feat_brand_cat
0.0064 ± 0.0007,feat_style_cat


In [86]:
df['brand'].value_counts(normalize=True)

nike                  0.097210
puma                  0.033315
ralph lauren          0.028775
vans                  0.021116
new balance           0.020295
                        ...   
lloyd                 0.000055
deadman wonderland    0.000055
nike air jordan i     0.000055
sessions              0.000055
zlyc                  0.000055
Name: brand, Length: 1732, dtype: float64

In [95]:
df[ df[ 'brand'] == 'nike'].features_parsed.sample(5).values

array([{},
       {'material': 'canvas', 'gender': 'men', 'color': 'blue', 'model': '642290 412', 'manufacturer part number': '642290 412', 'brand': 'nike', 'age group': 'adult'},
       {'material': 'leather', 'gender': 'men', 'size': '11.5', 'shoe category': 'athletic & sneakers', 'fabric content': '100% leather', 'color': 'black/gym red -white', 'model': '832816-001', 'casual & dress shoe style': 'basketball shoes', 'manufacturer part number': '832816-001', 'brand': 'nike', 'age group': 'adult'},
       {'gender': 'men', 'brand': 'nike', 'fabric content': '100% leather', 'color': 'dark cayenne/university gold'},
       {'sport': 'football', 'main color': 'purple & green', 'type': 'cleats'}],
      dtype=object)

In [105]:
cd '/content/drive/My Drive/Colab Notebooks/dw_Matrix/matrix_one/'

/content/drive/My Drive/Colab Notebooks/dw_Matrix/matrix_one


In [106]:
ls

day3.ipynb  day4.ipynb


In [104]:
!git add 'day5.ipynb'

fatal: pathspec 'day5.ipynb' did not match any files
